# AlphaSnake — Entrenamiento AlphaZero para Snake

Pipeline completo de entrenamiento: **MCTS + Policy/Value ResNet** para Snake 10x10.

- **Red**: ResNet 6 bloques residuales (64 ch), policy head (softmax 4) + value head (tanh)
- **MCTS**: 400 sims, UCB c_puct=1.0, Dirichlet noise, food stochasticity sampling
- **Pipeline**: Self-play → Replay Buffer → Train → Evaluate → Champion update
- **Export**: ONNX para inferencia en navegador

**Requisitos**: GPU (Colab: Runtime → Change runtime type → T4 GPU | Vast.ai: PyTorch Jupyter con GPU).  
Con el notebook en el **directorio raíz**, los checkpoints se guardan en `./alphasnake`.

In [ ]:
# ============================================================
# Cell 1: Setup — Verificar GPU y directorio de checkpoints
# ============================================================
# Notebook en directorio raíz: checkpoints van a SAVE_DIR (subcarpeta 'alphasnake').
# Colab: Drive. Vast.ai / local: <raíz>/alphasnake (ej. /workspace/alphasnake).
import os
import torch

# Directorio base = donde se ejecuta el notebook (raíz del proyecto)
ROOT_DIR = os.getcwd()
if os.path.exists("/workspace"):
    # Vast.ai: persistente; notebook en raíz -> /workspace/alphasnake
    SAVE_DIR = os.path.join(ROOT_DIR, "alphasnake")
elif os.path.exists("/content"):
    try:
        from google.colab import drive
        drive.mount("/content/drive")
        SAVE_DIR = "/content/drive/MyDrive/alphasnake"
    except ImportError:
        SAVE_DIR = "/content/alphasnake"
else:
    SAVE_DIR = os.path.join(ROOT_DIR, "alphasnake")

os.makedirs(SAVE_DIR, exist_ok=True)

print(f"PyTorch {torch.__version__} | CUDA: {torch.cuda.is_available()} | Checkpoints: {SAVE_DIR}")

### Progreso del entrenamiento
El entrenamiento muestra por iteración: self-play (wins, scores, ejemplos), training (loss), evaluación y resumen con tiempos y ETA.

In [ ]:
# La salida del entrenamiento muestra progreso detallado por iteración (self-play, training, eval, ETA).
# No se imprime uso de CPU/GPU para mantener el foco en métricas del modelo.
pass

In [ ]:
# ============================================================
# Cell 2: Imports y Configuracion
# ============================================================
import numpy as np
import random
import math
import time
import pickle
import subprocess
import threading
import multiprocessing
from queue import Empty
import dataclasses
from dataclasses import dataclass
from collections import deque
from tqdm.auto import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F

# -----------------------------------------------------------
# Configuracion — Ajustar segun recursos disponibles
# "rtx4060" = RTX 4060 Ti + ~5 nucleos (200 sims, 300 juegos, batch 64) — recomendado para Vast.ai
# "paper"   = settings del paper (~2h+ por iter, 400 sims, 1000 juegos)
# "fast"    = iter ~15-30 min (100 sims, 200 juegos)
# "colab_free" = muy reducido para pruebas rapidas
# -----------------------------------------------------------
PRESET = "paper"  # "paper" | "rtx4060" (4060 Ti + 5 nucleos) | "fast" | "colab_free"

@dataclass
class Config:
    # Entorno
    board_size: int = 10
    max_steps: int = 1000

    # Red neuronal
    net_channels: int = 64
    net_blocks: int = 6

    # MCTS
    num_simulations: int = 400
    c_puct: float = 1.0
    dir_alpha: float = 0.03
    dir_eps: float = 0.25
    temp_init: float = 1.0
    temp_final: float = 0.0
    temp_decay_move: int = 30
    food_samples: int = 8

    # Training
    lr: float = 1e-3
    weight_decay: float = 1e-4
    batch_size: int = 128
    buffer_size: int = 200_000
    epochs_per_iter: int = 10

    # Self-play
    games_per_iter: int = 1000
    max_iterations: int = 200

    # Evaluacion
    eval_games: int = 200
    accept_threshold: float = 0.55

    # Checkpointing
    save_dir: str = SAVE_DIR
    checkpoint_interval: int = 1

    # Rendimiento
    selfplay_workers: int = 5
    inference_batch_size: int = 64
    inference_timeout_ms: int = 20
    use_amp: bool = True

    # Logging de progreso
    verbose_game_updates: bool = True
    game_log_interval: int = 1
    game_tick_moves: int = 100
    heartbeat_seconds: int = 30

if PRESET == "rtx4060":
    # RTX 4060 Ti (8GB) con CPU multi-core: priorizar throughput estable sin OOM.
    config = Config(
        num_simulations=220,
        games_per_iter=360,
        eval_games=80,
        max_iterations=200,
        batch_size=128,
        buffer_size=140_000,
        food_samples=6,
        selfplay_workers=10,
        inference_batch_size=128,
        inference_timeout_ms=8,
        use_amp=True,
    )
    print("Usando preset RTX4060 optimizado (220 sims, 360 juegos/iter, batch 128, 10 workers)")
elif PRESET == "colab_free":
    config = Config(
        num_simulations=100,
        games_per_iter=50,
        eval_games=30,
        max_iterations=100,
        buffer_size=50_000,
        food_samples=4,
    )
    print("Usando preset COLAB_FREE (reducido para pruebas rapidas)")
elif PRESET == "fast":
    config = Config(
        num_simulations=100,
        games_per_iter=200,
        eval_games=50,
        max_iterations=200,
        buffer_size=100_000,
        food_samples=4,
    )
    print("Usando preset FAST (~100 sims, 200 juegos/iter; iteracion ~20-45 min)")
else:
    config = Config()
    print("Usando preset PAPER (400 sims, 1000 juegos/iter; requiere mucho tiempo)")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

if torch.cuda.is_available():
    torch.backends.cudnn.benchmark = True
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True
    torch.set_float32_matmul_precision("high")

print(f"Device: {device}")
print(f"Config: {dataclasses.asdict(config)}")

## Entorno Snake

Grid 10x10, observacion `(4, 10, 10)`:
- Canal 0: cuerpo de la serpiente
- Canal 1: cabeza
- Canal 2: comida
- Canal 3: direccion actual (valor constante normalizado en todo el plano)

Recompensas: `+1` comida, `-1` muerte, `0` otro. Max 1000 pasos.

In [ ]:
# ============================================================
# Cell 3: Entorno Snake
# ============================================================

class SnakeEnv:
    """
    Snake environment estilo AlphaSnake.
    Grid board_size x board_size, 4 acciones, observacion (4, H, W).
    """
    # 0=UP, 1=DOWN, 2=LEFT, 3=RIGHT
    ACTIONS = {0: (0, -1), 1: (0, 1), 2: (-1, 0), 3: (1, 0)}
    OPPOSITES = {0: 1, 1: 0, 2: 3, 3: 2}
    ACTION_NAMES = {0: 'UP', 1: 'DOWN', 2: 'LEFT', 3: 'RIGHT'}

    def __init__(self, board_size=10, max_steps=1000):
        self.board_size = board_size
        self.max_steps = max_steps
        self.max_score = board_size * board_size - 3  # 97 para 10x10
        self.reset()

    def reset(self):
        mid = self.board_size // 2
        # Snake empieza en el centro, apuntando a la derecha
        self.snake = deque([(mid, mid), (mid - 1, mid), (mid - 2, mid)])
        self.direction = 3  # RIGHT
        self.food = None
        self.done = False
        self.steps = 0
        self.score = 0
        self._place_food()
        return self.get_state()

    def _get_free_cells(self):
        snake_set = set(self.snake)
        return [(x, y) for y in range(self.board_size)
                for x in range(self.board_size)
                if (x, y) not in snake_set]

    def _place_food(self):
        free = self._get_free_cells()
        if free:
            self.food = random.choice(free)
        else:
            # Tablero lleno = victoria
            self.food = None
            self.done = True

    def get_state(self):
        """Retorna observacion (4, board_size, board_size) float32."""
        s = np.zeros((4, self.board_size, self.board_size), dtype=np.float32)
        # Canal 0: cuerpo
        for x, y in self.snake:
            s[0, y, x] = 1.0
        # Canal 1: cabeza
        hx, hy = self.snake[0]
        s[1, hy, hx] = 1.0
        # Canal 2: comida
        if self.food:
            fx, fy = self.food
            s[2, fy, fx] = 1.0
        # Canal 3: direccion (valor constante normalizado)
        # UP=0.25, DOWN=0.5, LEFT=0.75, RIGHT=1.0
        s[3, :, :] = (self.direction + 1) / 4.0
        return s

    def valid_actions(self):
        """Acciones validas (excluye reversa directa)."""
        rev = self.OPPOSITES[self.direction]
        return [a for a in range(4) if a != rev]

    def step(self, action):
        """
        Ejecuta accion. Retorna (reward, done).
        reward: +1.0 comida, -1.0 muerte, 0.0 otro.
        """
        if self.done:
            return 0.0, True

        # Prohibir reversa directa: si intenta reversa, mantener direccion
        if action == self.OPPOSITES[self.direction]:
            action = self.direction

        self.direction = action
        dx, dy = self.ACTIONS[action]
        hx, hy = self.snake[0]
        nx, ny = hx + dx, hy + dy

        # Colision con pared
        if nx < 0 or nx >= self.board_size or ny < 0 or ny >= self.board_size:
            self.done = True
            return -1.0, True

        # Verificar si comera comida (antes de mover)
        ate_food = self.food is not None and (nx, ny) == self.food

        # Colision con cuerpo
        # Si NO come comida, la cola se movera, asi que es valido ir a la pos actual de la cola
        body_set = set(self.snake)
        if not ate_food:
            body_set.discard(self.snake[-1])
        if (nx, ny) in body_set:
            self.done = True
            return -1.0, True

        # Mover
        self.snake.appendleft((nx, ny))
        if ate_food:
            self.score += 1
            self._place_food()
            if self.done:  # _place_food puso done=True si el tablero esta lleno
                return 1.0, True
        else:
            self.snake.pop()

        self.steps += 1
        if self.steps >= self.max_steps:
            self.done = True
            return 0.0, True

        return (1.0 if ate_food else 0.0), False

    def is_win(self):
        """True si la serpiente lleno todo el tablero."""
        return len(self.snake) >= self.board_size ** 2

    def clone(self):
        """Copia eficiente del entorno."""
        env = SnakeEnv.__new__(SnakeEnv)
        env.board_size = self.board_size
        env.max_steps = self.max_steps
        env.max_score = self.max_score
        env.snake = deque(self.snake)
        env.direction = self.direction
        env.food = self.food
        env.done = self.done
        env.steps = self.steps
        env.score = self.score
        return env

# --- Test rapido ---
env = SnakeEnv(10, 1000)
s = env.reset()
print(f"State shape: {s.shape}")
print(f"Snake length: {len(env.snake)}, food: {env.food}")
print(f"Valid actions: {[env.ACTION_NAMES[a] for a in env.valid_actions()]}")

# Jugar unos pasos random
for _ in range(20):
    a = random.choice(env.valid_actions())
    r, done = env.step(a)
    if done:
        print(f"Game over! Score: {env.score}, Win: {env.is_win()}")
        break
else:
    print(f"After 20 steps: score={env.score}, length={len(env.snake)}")

## Red Neuronal — AlphaSnakeNet

ResNet estilo AlphaZero reducido:
- **Stem**: Conv2d(4→64, 3x3) + BN + ReLU
- **6 bloques residuales**: Conv(64, 3x3) → BN → ReLU → Conv(64, 3x3) → BN → skip → ReLU
- **Policy head**: Conv(2, 1x1) → BN → ReLU → Flatten → Linear(200→4) → Softmax
- **Value head**: Conv(1, 1x1) → BN → ReLU → Flatten → Linear(100→64) → ReLU → Linear(64→1) → Tanh

In [ ]:
# ============================================================
# Cell 4: Red Neuronal — AlphaSnakeNet
# ============================================================

class ResBlock(nn.Module):
    """Bloque residual: Conv→BN→ReLU→Conv→BN→skip→ReLU"""
    def __init__(self, channels):
        super().__init__()
        self.conv1 = nn.Conv2d(channels, channels, 3, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(channels)
        self.conv2 = nn.Conv2d(channels, channels, 3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(channels)

    def forward(self, x):
        residual = x
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out = F.relu(out + residual)
        return out


class AlphaSnakeNet(nn.Module):
    """
    Policy + Value network estilo AlphaZero.
    Input:  (batch, 4, board_size, board_size)
    Output: policy (batch, 4), value (batch, 1)
    """
    def __init__(self, in_channels=4, num_blocks=6, channels=64, board_size=10):
        super().__init__()
        self.board_size = board_size

        # Stem
        self.stem = nn.Sequential(
            nn.Conv2d(in_channels, channels, 3, padding=1, bias=False),
            nn.BatchNorm2d(channels),
            nn.ReLU()
        )

        # Residual tower
        self.res_tower = nn.Sequential(*[ResBlock(channels) for _ in range(num_blocks)])

        # Policy head
        self.policy_conv = nn.Conv2d(channels, 2, 1, bias=False)
        self.policy_bn = nn.BatchNorm2d(2)
        self.policy_fc = nn.Linear(2 * board_size * board_size, 4)

        # Value head
        self.value_conv = nn.Conv2d(channels, 1, 1, bias=False)
        self.value_bn = nn.BatchNorm2d(1)
        self.value_fc1 = nn.Linear(board_size * board_size, 64)
        self.value_fc2 = nn.Linear(64, 1)

    def forward(self, x):
        # Shared trunk
        x = self.stem(x)
        x = self.res_tower(x)

        # Policy
        p = F.relu(self.policy_bn(self.policy_conv(x)))
        p = p.view(p.size(0), -1)
        p = F.softmax(self.policy_fc(p), dim=1)

        # Value
        v = F.relu(self.value_bn(self.value_conv(x)))
        v = v.view(v.size(0), -1)
        v = F.relu(self.value_fc1(v))
        v = torch.tanh(self.value_fc2(v))

        return p, v

# --- Test ---
net = AlphaSnakeNet(
    in_channels=4,
    num_blocks=config.net_blocks,
    channels=config.net_channels,
    board_size=config.board_size
).to(device)

total_params = sum(p.numel() for p in net.parameters())
print(f"AlphaSnakeNet: {total_params:,} parametros")

dummy = torch.randn(1, 4, 10, 10).to(device)
pol, val = net(dummy)
print(f"Policy shape: {pol.shape}, sum={pol.sum().item():.4f}")
print(f"Value shape: {val.shape}, val={val.item():.4f}")
del net, dummy  # liberar memoria

## MCTS — Monte Carlo Tree Search

Implementacion AlphaZero con manejo de estocasticidad (food sampling).

- UCB: `Q(s,a) + c_puct * P(s,a) * sqrt(N(s)) / (1 + N(s,a))`
- Dirichlet noise en root: `P = (1 - eps) * P + eps * Dir(alpha)`
- Food stochasticity: al expandir nodo donde se comio comida, promediar valor sobre k posiciones de comida

In [ ]:
# ============================================================
# Cell 5: MCTS — Monte Carlo Tree Search
# ============================================================

class MCTSNode:
    """Nodo del arbol MCTS."""
    __slots__ = ['prior', 'visit_count', 'value_sum', 'children',
                 'is_expanded', 'env', 'food_eaten']

    def __init__(self, prior=0.0):
        self.prior = prior
        self.visit_count = 0
        self.value_sum = 0.0
        self.children = {}        # action (int) -> MCTSNode
        self.is_expanded = False
        self.env = None           # SnakeEnv snapshot
        self.food_eaten = False   # True si la transicion a este nodo comio comida

    def value(self):
        if self.visit_count == 0:
            return 0.0
        return self.value_sum / self.visit_count


class MCTS:
    """
    Monte Carlo Tree Search estilo AlphaZero con soporte
    para estocasticidad de comida (food sampling).
    Acepta predict_fn y predict_batch_fn para permitir
    batching de inferencia desde multiples hilos.
    """
    def __init__(self, predict_fn, cfg, predict_batch_fn=None):
        self.predict_fn = predict_fn
        self.predict_batch_fn = predict_batch_fn or (lambda states: [predict_fn(s) for s in states])
        self.c_puct = cfg.c_puct
        self.num_simulations = cfg.num_simulations
        self.dir_alpha = cfg.dir_alpha
        self.dir_eps = cfg.dir_eps
        self.food_samples = cfg.food_samples

    def _predict(self, state_np):
        """Inferir policy y value."""
        return self.predict_fn(state_np)

    def _ucb_score(self, parent, child):
        """UCB score clasico AlphaZero."""
        q = child.value()
        u = self.c_puct * child.prior * math.sqrt(parent.visit_count) / (1 + child.visit_count)
        return q + u

    def _select_child(self, node):
        """Seleccionar hijo con mayor UCB score."""
        best_score = -float('inf')
        best_action = -1
        best_child = None
        for action, child in node.children.items():
            score = self._ucb_score(node, child)
            if score > best_score:
                best_score = score
                best_action = action
                best_child = child
        return best_action, best_child

    def _expand(self, node):
        """
        Expandir nodo hoja: evaluar con la red, crear hijos.
        Maneja estocasticidad de comida promediando valores.
        Retorna el valor del nodo.
        """
        env = node.env
        if env.done:
            return 1.0 if env.is_win() else -1.0

        policy, value = self._predict(env.get_state())

        # --- Food stochasticity (batched via predict_batch_fn) ---
        if node.food_eaten and self.food_samples > 1:
            snake_set = set(env.snake)
            free = [(x, y) for y in range(env.board_size)
                    for x in range(env.board_size)
                    if (x, y) not in snake_set and (x, y) != env.food]
            k = min(self.food_samples - 1, len(free))
            if k > 0:
                sampled_positions = random.sample(free, k)
                food_states = []
                for food_pos in sampled_positions:
                    env_copy = env.clone()
                    env_copy.food = food_pos
                    food_states.append(env_copy.get_state())
                results = self.predict_batch_fn(food_states)
                food_values = [r[1] for r in results]
                value = (value + sum(food_values)) / (1 + len(food_values))

        # Mascara de acciones validas y renormalizacion
        valid = env.valid_actions()
        mask = np.zeros(4, dtype=np.float32)
        for a in valid:
            mask[a] = 1.0
        policy = policy * mask
        total = policy.sum()
        if total > 0:
            policy /= total
        else:
            policy = mask / mask.sum()

        # Crear hijos
        for a in valid:
            node.children[a] = MCTSNode(prior=policy[a])

        node.is_expanded = True
        return value

    def _add_dirichlet_noise(self, node):
        """Agregar Dirichlet noise al root para exploracion."""
        actions = list(node.children.keys())
        if not actions:
            return
        noise = np.random.dirichlet([self.dir_alpha] * len(actions))
        for i, a in enumerate(actions):
            node.children[a].prior = (
                (1 - self.dir_eps) * node.children[a].prior +
                self.dir_eps * noise[i]
            )

    def search(self, root_env, temperature=1.0):
        """
        Ejecutar MCTS completo desde un estado raiz.
        Retorna distribucion de probabilidad sobre acciones (4,).
        """
        root = MCTSNode()
        root.env = root_env.clone()

        # Expandir raiz
        self._expand(root)
        self._add_dirichlet_noise(root)

        # Simulaciones
        for _ in range(self.num_simulations):
            node = root
            path = [node]

            # --- SELECT ---
            while node.is_expanded and node.children:
                action, child = self._select_child(node)

                # Computacion lazy del estado del hijo
                if child.env is None:
                    env_copy = node.env.clone()
                    old_score = env_copy.score
                    env_copy.step(action)
                    child.env = env_copy
                    child.food_eaten = (env_copy.score > old_score and not env_copy.done)

                node = child
                path.append(node)

            # --- EXPAND & EVALUATE ---
            if not node.is_expanded:
                value = self._expand(node)
            else:
                # Nodo terminal (expanded pero sin hijos)
                value = 1.0 if node.env.is_win() else -1.0

            # --- BACKUP ---
            for n in reversed(path):
                n.visit_count += 1
                n.value_sum += value

        # --- Construir distribucion de acciones ---
        visits = np.zeros(4, dtype=np.float32)
        for a, child in root.children.items():
            visits[a] = child.visit_count

        if temperature == 0:
            # Greedy
            best = np.argmax(visits)
            probs = np.zeros(4, dtype=np.float32)
            probs[best] = 1.0
        else:
            # Con temperatura
            visits_temp = np.power(visits, 1.0 / temperature)
            total = visits_temp.sum()
            if total > 0:
                probs = visits_temp / total
            else:
                probs = np.ones(4, dtype=np.float32) / 4.0

        return probs

print("MCTS implementado correctamente.")

In [ ]:
# ============================================================
# Cell 5b: InferenceBatcher — Batching de GPU para self-play paralelo
# ============================================================
import queue as queue_module
import concurrent.futures

class InferenceBatcher:
    """
    Recolecta requests de inferencia de multiples hilos y los
    procesa en batches en la GPU. Esto maximiza la utilizacion
    de la GPU durante self-play paralelo.
    """
    def __init__(self, net, device, max_batch=64, timeout_ms=3):
        self.net = net
        self.device = device
        self.max_batch = max_batch
        self.timeout = timeout_ms / 1000.0
        self._queue = queue_module.Queue()
        self._running = False
        self._thread = None
        self._total_calls = 0
        self._total_batches = 0

    def start(self):
        self._running = True
        self._total_calls = 0
        self._total_batches = 0
        self._thread = threading.Thread(target=self._worker, daemon=True)
        self._thread.start()
        return self

    def stop(self):
        self._running = False
        self._queue.put(None)  # sentinel
        if self._thread:
            self._thread.join(timeout=10)

    def submit(self, state_np):
        """Non-blocking: enviar estado, retorna Future."""
        f = concurrent.futures.Future()
        self._queue.put((state_np, f))
        return f

    def predict(self, state_np):
        """Blocking: enviar estado, esperar resultado (policy, value)."""
        return self.submit(state_np).result()

    def predict_batch(self, states_list):
        """Enviar multiples estados, esperar todos los resultados."""
        if not states_list:
            return []
        futures = [self.submit(s) for s in states_list]
        return [f.result() for f in futures]

    def stats(self):
        avg = self._total_calls / max(self._total_batches, 1)
        return self._total_calls, self._total_batches, avg

    def _worker(self):
        while self._running:
            batch = []
            # Esperar primer request
            try:
                item = self._queue.get(timeout=1.0)
            except queue_module.Empty:
                continue
            if item is None:
                break
            batch.append(item)

            # Recolectar mas requests (hasta max_batch o timeout)
            deadline = time.time() + self.timeout
            while len(batch) < self.max_batch:
                remaining = deadline - time.time()
                if remaining <= 0:
                    break
                try:
                    item = self._queue.get(timeout=max(0.0001, remaining))
                except queue_module.Empty:
                    break
                if item is None:
                    self._running = False
                    break
                batch.append(item)

            if not batch:
                continue

            states = [b[0] for b in batch]
            futures_list = [b[1] for b in batch]
            self._total_calls += len(batch)
            self._total_batches += 1

            try:
                arr = np.stack(states)
                t = torch.as_tensor(arr, device=self.device)
                with torch.no_grad():
                    policies, values = self.net(t)
                p_np = policies.cpu().numpy()
                v_np = values.cpu().numpy()
                for i, f in enumerate(futures_list):
                    f.set_result((p_np[i], float(v_np[i, 0])))
            except Exception as e:
                for f in futures_list:
                    if not f.done():
                        f.set_exception(e)


def make_predict_fn(net, device):
    """Crear funcion de prediccion directa (para uso single-thread, eval)."""
    @torch.no_grad()
    def predict(state_np):
        t = torch.as_tensor(state_np, device=device).unsqueeze(0)
        p, v = net(t)
        return p[0].cpu().numpy(), v.item()
    return predict

print("InferenceBatcher implementado.")

## Self-Play, Replay Buffer, Training y Evaluacion

- **Self-play**: jugar partidas completas usando MCTS, guardar `(state, pi, z)` por cada paso
- **Replay Buffer**: buffer circular de 200k posiciones
- **Training**: Adam, loss = MSE(z, v) + CrossEntropy(pi, p) + L2 regularizacion
- **Evaluacion**: comparar modelos por win rate independiente

In [ ]:
# ============================================================
# Cell 6: Self-Play
# ============================================================

def self_play_game(predict_fn, cfg, predict_batch_fn=None, progress_cb=None):
    """
    Jugar una partida completa usando MCTS.
    predict_fn: callable(state_np) -> (policy_np, value_float)
    predict_batch_fn: callable(list[state_np]) -> list[(policy, value)]
    progress_cb: callable(move_count, score, length) opcional para telemetria.
    Retorna: (examples, is_win, score, num_moves)
    """
    env = SnakeEnv(cfg.board_size, cfg.max_steps)
    mcts = MCTS(predict_fn, cfg, predict_batch_fn)
    env.reset()

    states = []
    policies = []
    move_count = 0

    while not env.done:
        # Temperatura: 1.0 hasta move temp_decay_move, luego 0.0
        temp = cfg.temp_init if move_count < cfg.temp_decay_move else cfg.temp_final

        # Busqueda MCTS
        pi = mcts.search(env, temperature=temp)

        # Guardar datos de entrenamiento
        states.append(env.get_state().copy())
        policies.append(pi.copy())

        # Seleccionar accion
        if temp == 0:
            action = int(np.argmax(pi))
        else:
            action = int(np.random.choice(4, p=pi))

        # Ejecutar
        env.step(action)
        move_count += 1
        if progress_cb is not None:
            tick_every = max(25, int(getattr(cfg, "game_tick_moves", 100)))
            if move_count == 1 or move_count % tick_every == 0:
                progress_cb(move_count, env.score, len(env.snake))

    # Determinar resultado: +1 si gano (lleno tablero), -1 si no
    z = 1.0 if env.is_win() else -1.0

    # Crear ejemplos de entrenamiento
    examples = [(s, p, z) for s, p in zip(states, policies)]
    return examples, env.is_win(), env.score, move_count, len(env.snake)


def run_n_games(n, request_queue, response_queue, progress_queue, worker_id, cfg):
    """
    Ejecutar n partidas de self-play en un proceso worker.
    Usa request_queue/response_queue para obtener (policy, value) del proceso principal.
    Necesario para multiprocessing: cada worker usa su propio nucleo (sin GIL).
    """
    def predict(state_np):
        request_queue.put((worker_id, state_np))
        return response_queue.get()

    def predict_batch(states_list):
        return [predict(s) for s in states_list]

    all_examples = []
    wins = 0
    scores_list = []
    moves_list = []
    lengths_list = []

    if progress_queue is not None:
        progress_queue.put({"type": "worker_start", "worker": worker_id, "games": n})

    for game_idx in range(n):
        if progress_queue is not None:
            progress_queue.put({"type": "game_start", "worker": worker_id, "game_local": game_idx + 1})

        def _progress_cb(move_count, score_now, len_now):
            if progress_queue is not None:
                progress_queue.put({
                    "type": "game_tick",
                    "worker": worker_id,
                    "game_local": game_idx + 1,
                    "moves": move_count,
                    "score": score_now,
                    "length": len_now,
                })

        examples, won, score, moves, length = self_play_game(predict, cfg, predict_batch, progress_cb=_progress_cb)
        all_examples.extend(examples)
        if won:
            wins += 1
        scores_list.append(score)
        moves_list.append(moves)
        lengths_list.append(length)

        if progress_queue is not None:
            progress_queue.put({
                "type": "game_end",
                "worker": worker_id,
                "game_local": game_idx + 1,
                "won": won,
                "score": score,
                "moves": moves,
                "length": length,
                "examples": len(examples),
            })

    return all_examples, wins, scores_list, moves_list, lengths_list


print("Self-play implementado.")

In [ ]:
# ============================================================
# Cell 7: Replay Buffer y Training
# ============================================================

class ReplayBuffer:
    """Buffer circular para almacenar ejemplos de self-play."""
    def __init__(self, max_size):
        self.buffer = deque(maxlen=max_size)

    def add(self, examples):
        """Agregar lista de (state, policy, value) al buffer."""
        self.buffer.extend(examples)

    def sample(self, batch_size):
        """Samplear un batch aleatorio."""
        n = min(batch_size, len(self.buffer))
        indices = np.random.choice(len(self.buffer), size=n, replace=False)
        batch = [self.buffer[i] for i in indices]
        states = np.array([b[0] for b in batch])
        policies = np.array([b[1] for b in batch])
        values = np.array([b[2] for b in batch], dtype=np.float32)
        return states, policies, values

    def __len__(self):
        return len(self.buffer)

    def save(self, path):
        with open(path, 'wb') as f:
            pickle.dump(list(self.buffer), f)

    def load(self, path):
        if os.path.exists(path):
            with open(path, 'rb') as f:
                data = pickle.load(f)
            self.buffer = deque(data, maxlen=self.buffer.maxlen)
            return True
        return False


def train_epoch(net, optimizer, buffer, cfg, device, scaler=None):
    """
    Entrenar una epoca sobre el replay buffer.
    Loss = (z - v)^2 - pi * log(p)
    (weight_decay ya esta en el optimizer como L2)
    """
    net.train()
    total_loss = 0.0
    total_p_loss = 0.0
    total_v_loss = 0.0
    n_batches = 0

    num_batches = max(1, len(buffer) // cfg.batch_size)
    use_amp = bool(getattr(cfg, "use_amp", True) and device.type == "cuda")

    for _ in range(num_batches):
        states, policies, values = buffer.sample(cfg.batch_size)

        states_t = torch.from_numpy(states)
        policies_t = torch.from_numpy(policies)
        values_t = torch.from_numpy(values).unsqueeze(1)

        if device.type == "cuda":
            states_t = states_t.pin_memory().to(device, non_blocking=True)
            policies_t = policies_t.pin_memory().to(device, non_blocking=True)
            values_t = values_t.pin_memory().to(device, non_blocking=True)
        else:
            states_t = states_t.to(device)
            policies_t = policies_t.to(device)
            values_t = values_t.to(device)

        with torch.autocast(device_type="cuda", dtype=torch.float16, enabled=use_amp):
            pred_p, pred_v = net(states_t)

            # Value loss: MSE
            v_loss = F.mse_loss(pred_v, values_t)

            # Policy loss: cross-entropy (pi * log(p))
            p_loss = -torch.mean(torch.sum(policies_t * torch.log(pred_p + 1e-8), dim=1))

            loss = v_loss + p_loss

        optimizer.zero_grad(set_to_none=True)
        if use_amp and scaler is not None:
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            loss.backward()
            optimizer.step()

        total_loss += loss.item()
        total_p_loss += p_loss.item()
        total_v_loss += v_loss.item()
        n_batches += 1

    if n_batches == 0:
        return 0, 0, 0
    return total_loss / n_batches, total_p_loss / n_batches, total_v_loss / n_batches

print("ReplayBuffer y train_epoch implementados.")

In [ ]:
# ============================================================
# Cell 8: Evaluacion
# ============================================================

def evaluate_mcts(net, cfg, device, n_games=None):
    """
    Evaluar modelo usando MCTS (greedy, temp=0).
    Retorna (win_rate, avg_score, scores).
    """
    if n_games is None:
        n_games = cfg.eval_games
    net.eval()

    wins = 0
    total_score = 0
    scores = []

    for i in tqdm(range(n_games), desc="Eval MCTS", leave=False):
        env = SnakeEnv(cfg.board_size, cfg.max_steps)
        mcts_eval = MCTS(make_predict_fn(net, device), cfg)
        env.reset()

        while not env.done:
            pi = mcts_eval.search(env, temperature=0)
            action = int(np.argmax(pi))
            env.step(action)

        if env.is_win():
            wins += 1
        total_score += env.score
        scores.append(env.score)

    wr = wins / max(n_games, 1)
    avg = total_score / max(n_games, 1)
    return wr, avg, scores


def evaluate_policy_only(net, cfg, device, n_games=200):
    """
    Evaluacion rapida usando solo la policy network (sin MCTS).
    Util para monitoreo rapido entre iteraciones.
    """
    net.eval()
    wins = 0
    total_score = 0

    for _ in tqdm(range(n_games), desc="Eval Policy", leave=False):
        env = SnakeEnv(cfg.board_size, cfg.max_steps)
        env.reset()

        while not env.done:
            state_t = torch.from_numpy(env.get_state()).unsqueeze(0).to(device)
            with torch.no_grad():
                policy, _ = net(state_t)

            # Mascara de acciones validas
            valid = env.valid_actions()
            mask = torch.zeros(4, device=device)
            for a in valid:
                mask[a] = 1.0
            policy = policy.squeeze() * mask
            total = policy.sum()
            if total > 0:
                policy = policy / total
            else:
                policy = mask / mask.sum()

            action = int(torch.argmax(policy).item())
            env.step(action)

        if env.is_win():
            wins += 1
        total_score += env.score

    wr = wins / max(n_games, 1)
    avg = total_score / max(n_games, 1)
    return wr, avg

print("Funciones de evaluacion implementadas.")

## Loop Principal de Entrenamiento

Ciclo completo: Self-play → Train → Evaluate → Champion update.
Guarda checkpoints en `save_dir` (Colab: Drive; Vast.ai: /workspace o volumen) para recuperar de desconexiones.

In [ ]:
# ============================================================
# Cell 9: Loop Principal de Entrenamiento
# ============================================================

def save_checkpoint(net, best_net, optimizer, iteration, best_win_rate, cfg, buffer):
    """Guardar checkpoint completo en save_dir."""
    ckpt = {
        'iteration': iteration,
        'net': net.state_dict(),
        'best_net': best_net.state_dict(),
        'optimizer': optimizer.state_dict(),
        'best_win_rate': best_win_rate,
        'config': dataclasses.asdict(cfg),
    }
    path = os.path.join(cfg.save_dir, 'latest_checkpoint.pt')
    torch.save(ckpt, path)

    # Guardar mejor modelo por separado
    best_path = os.path.join(cfg.save_dir, 'best_model.pt')
    torch.save(best_net.state_dict(), best_path)

    # Guardar buffer
    buffer_path = os.path.join(cfg.save_dir, 'buffer.pkl')
    buffer.save(buffer_path)

    print(f"  Checkpoint guardado (iter {iteration})")


def load_checkpoint(net, best_net, optimizer, cfg, buffer):
    """Cargar checkpoint desde save_dir si existe."""
    path = os.path.join(cfg.save_dir, 'latest_checkpoint.pt')
    if not os.path.exists(path):
        return 0, 0.0

    ckpt = torch.load(path, map_location=device)
    net.load_state_dict(ckpt['net'])
    best_net.load_state_dict(ckpt['best_net'])
    optimizer.load_state_dict(ckpt['optimizer'])

    buffer_path = os.path.join(cfg.save_dir, 'buffer.pkl')
    buffer.load(buffer_path)

    iteration = ckpt.get('iteration', 0)
    best_wr = ckpt.get('best_win_rate', 0.0)
    print(f"Checkpoint cargado: iteracion {iteration}, best_win_rate={best_wr:.3f}")
    print(f"Buffer restaurado: {len(buffer)} ejemplos")
    return iteration, best_wr


def train_alphasnake(cfg):
    """Loop principal de entrenamiento AlphaSnake."""
    print(f"{'='*60}")
    print(f" AlphaSnake Training")
    print(f" Board: {cfg.board_size}x{cfg.board_size}")
    print(f" Simulations: {cfg.num_simulations}")
    print(f" Games/iter: {cfg.games_per_iter}")
    print(f"{'='*60}\n")

    # Crear redes
    net = AlphaSnakeNet(4, cfg.net_blocks, cfg.net_channels, cfg.board_size).to(device)
    best_net = AlphaSnakeNet(4, cfg.net_blocks, cfg.net_channels, cfg.board_size).to(device)
    best_net.load_state_dict(net.state_dict())

    optimizer = torch.optim.Adam(net.parameters(), lr=cfg.lr, weight_decay=cfg.weight_decay)
    buffer = ReplayBuffer(cfg.buffer_size)
    scaler = torch.amp.GradScaler(device="cuda", enabled=bool(getattr(cfg, "use_amp", True) and device.type == "cuda"))

    # Intentar cargar checkpoint
    start_iter, best_win_rate = load_checkpoint(net, best_net, optimizer, cfg, buffer)

    training_start = time.time()
    for iteration in range(start_iter, cfg.max_iterations):
        iter_start = time.time()
        print(f"\n{'='*60}")
        print(f"  ITERACION {iteration + 1} / {cfg.max_iterations}")
        print(f"{'='*60}")

        # ------------------------------------------------
        # 1. SELF-PLAY PARALELO con procesos (usa todos los nucleos; evita GIL)
        # ------------------------------------------------
        cpu_total = os.cpu_count() or 8
        n_workers = max(1, min(int(getattr(cfg, "selfplay_workers", 5)), cpu_total))
        best_net.eval()
        all_examples = []
        sp_wins = 0
        sp_scores = []
        sp_moves = []
        sp_lengths = []

        # Manager().Queue() es pasable a procesos con spawn (Vast.ai/Jupyter); Queue() solo por herencia (fork)
        mp_manager = multiprocessing.Manager()
        mp_request_queue = mp_manager.Queue()
        response_queues = [mp_manager.Queue() for _ in range(n_workers)]
        progress_queue = mp_manager.Queue()

        def inference_server():
            batch_timeout = max(0.001, float(getattr(cfg, "inference_timeout_ms", 20)) / 1000.0)
            max_batch = max(8, int(getattr(cfg, "inference_batch_size", 64)))
            use_amp = bool(getattr(cfg, "use_amp", True) and device.type == "cuda")
            while True:
                batch = []
                deadline = time.time() + batch_timeout
                while len(batch) < max_batch:
                    try:
                        item = mp_request_queue.get(timeout=max(0.001, deadline - time.time()))
                        if item is None:
                            return
                        batch.append(item)
                    except Empty:
                        break
                if not batch:
                    continue
                wids = [b[0] for b in batch]
                states = [b[1] for b in batch]
                arr = np.stack(states)
                t = torch.from_numpy(arr)
                if device.type == "cuda":
                    t = t.pin_memory().to(device, non_blocking=True)
                else:
                    t = t.to(device)
                with torch.no_grad():
                    with torch.autocast(device_type="cuda", dtype=torch.float16, enabled=use_amp):
                        policies, values = best_net(t)
                p_np = policies.cpu().numpy()
                v_np = values.cpu().numpy()
                for i in range(len(batch)):
                    response_queues[wids[i]].put((p_np[i], float(v_np[i, 0])))

        server_thread = threading.Thread(target=inference_server, daemon=True)
        server_thread.start()

        iter_stamp = time.strftime("%Y-%m-%d %H:%M:%S")
        print(f"  [Iter {iteration + 1}] Inicio confirmado: {iter_stamp}")
        print(f"  [Self-play] Arranco con {n_workers} workers y {cfg.games_per_iter} juegos objetivo")

        sp_start = time.time()
        games_per_worker = [cfg.games_per_iter // n_workers + (1 if i < cfg.games_per_iter % n_workers else 0) for i in range(n_workers)]

        with concurrent.futures.ProcessPoolExecutor(max_workers=n_workers) as pool:
            futures = [
                pool.submit(run_n_games, games_per_worker[i], mp_request_queue, response_queues[i], progress_queue, i, cfg)
                for i in range(n_workers)
            ]
            pending = set(futures)
            completed = 0
            games_reported = 0
            verbose_games = bool(getattr(cfg, "verbose_game_updates", True))
            log_every = max(1, int(getattr(cfg, "game_log_interval", 1)))
            heartbeat_s = max(10, int(getattr(cfg, "heartbeat_seconds", 30)))
            last_heartbeat = time.time()

            with tqdm(total=cfg.games_per_iter, desc=f"Self-play iter {iteration+1}", mininterval=0.5, dynamic_ncols=True) as pbar:
                while pending:
                    drained = 0
                    while True:
                        try:
                            msg = progress_queue.get_nowait()
                        except Empty:
                            break

                        drained += 1
                        msg_type = msg.get("type", "unknown") if isinstance(msg, dict) else "unknown"

                        if msg_type == "worker_start":
                            print(f"      [Worker {msg['worker']}] iniciado | juegos_asignados={msg['games']}", flush=True)
                            continue

                        if msg_type == "game_start":
                            print(f"      [Worker {msg['worker']}] inicio juego local={msg['game_local']}", flush=True)
                            continue

                        if msg_type == "game_tick":
                            if verbose_games:
                                print(
                                    f"      [Tick] worker={msg['worker']} juego={msg['game_local']} "
                                    f"moves={msg['moves']} score={msg['score']} len={msg['length']}",
                                    flush=True,
                                )
                            continue

                        if msg_type == "game_end":
                            games_reported += 1
                            pbar.update(1)
                            if verbose_games and (games_reported == 1 or games_reported % log_every == 0):
                                result_tag = "WIN" if msg["won"] else "LOSE"
                                print(
                                    f"      [Juego {games_reported}/{cfg.games_per_iter}] "
                                    f"worker={msg['worker']} local={msg['game_local']} {result_tag} "
                                    f"score={msg['score']} moves={msg['moves']} len={msg['length']} ex={msg['examples']}",
                                    flush=True,
                                )
                            continue

                    done_now, pending = concurrent.futures.wait(
                        pending,
                        timeout=1.0,
                        return_when=concurrent.futures.FIRST_COMPLETED,
                    )

                    if not done_now:
                        now = time.time()
                        if drained == 0 and now - last_heartbeat >= heartbeat_s:
                            print(
                                f"      [Heartbeat] esperando progreso | juegos={games_reported}/{cfg.games_per_iter} | workers listos={completed}/{n_workers} (siguiente tick/juego cuando avance MCTS)",
                                flush=True,
                            )
                            last_heartbeat = now
                        continue

                    for future in done_now:
                        ex, w, sc, mv, ln = future.result()
                        all_examples.extend(ex)
                        sp_wins += w
                        sp_scores.extend(sc)
                        sp_moves.extend(mv)
                        sp_lengths.extend(ln)
                        completed += 1
                        games_done = len(sp_scores)
                        wr_so_far = 100 * sp_wins / games_done if games_done else 0
                        avg_sc = np.mean(sp_scores) if sp_scores else 0
                        print(f"      [Worker completado] {completed}/{n_workers} | juegos agregados={len(sc)} | acumulado={games_done}/{cfg.games_per_iter} | wins={sp_wins} ({wr_so_far:.1f}%) | avg_score={avg_sc:.1f} | ejemplos={len(all_examples)}", flush=True)

        for _ in range(n_workers):
            mp_request_queue.put(None)
        server_thread.join(timeout=10)

        sp_time = time.time() - sp_start
        sp_wr = sp_wins / cfg.games_per_iter
        sp_scores_arr = np.array(sp_scores)
        sp_moves_arr = np.array(sp_moves)
        sp_lengths_arr = np.array(sp_lengths)

        ejemplos_per_min = len(all_examples) / (sp_time / 60.0) if sp_time > 0 else 0
        print(f"  --- Self-play ({sp_time:.0f}s, {n_workers} procesos) ---")
        print(f"    Juegos: {cfg.games_per_iter} | Wins: {sp_wins} ({100*sp_wr:.1f}%) | Ejemplos: {len(all_examples)} ({ejemplos_per_min:.0f}/min)")
        print(f"    Score  -> min: {sp_scores_arr.min():.0f}  med: {np.median(sp_scores_arr):.0f}  max: {sp_scores_arr.max():.0f}  avg: {sp_scores_arr.mean():.1f}")
        print(f"    Length -> min: {sp_lengths_arr.min():.0f}  med: {np.median(sp_lengths_arr):.0f}  max: {sp_lengths_arr.max():.0f}  (max: {cfg.board_size**2})")
        print(f"    Moves  -> min: {sp_moves_arr.min():.0f}  med: {np.median(sp_moves_arr):.0f}  max: {sp_moves_arr.max():.0f}")

        buffer.add(all_examples)
        print(f"    Buffer total: {len(buffer)}")

        # ------------------------------------------------
        # 2. TRAINING
        # ------------------------------------------------
        train_start = time.time()
        last_loss, last_p, last_v = 0.0, 0.0, 0.0
        for epoch in tqdm(range(cfg.epochs_per_iter), desc=f"Training iter {iteration+1}", leave=False):
            last_loss, last_p, last_v = train_epoch(net, optimizer, buffer, cfg, device, scaler=scaler)
            if (epoch + 1) % 5 == 0 or epoch == 0:
                tqdm.write(f"    Epoch {epoch+1}/{cfg.epochs_per_iter}: loss={last_loss:.4f} (policy={last_p:.4f}, value={last_v:.4f})")
        train_time = time.time() - train_start
        print(f"  --- Training ({train_time:.0f}s, {cfg.epochs_per_iter} epochs) ---")
        print(f"    Loss: total={last_loss:.4f} | policy={last_p:.4f} | value={last_v:.4f}")

        # ------------------------------------------------
        # 3. EVALUACION RAPIDA (policy-only para velocidad)
        # ------------------------------------------------
        eval_start = time.time()
        wr_new_pol, avg_new_pol = evaluate_policy_only(net, cfg, device, n_games=50)
        wr_best_pol, avg_best_pol = evaluate_policy_only(best_net, cfg, device, n_games=50)
        eval_time = time.time() - eval_start

        print(f"  --- Eval policy-only (50 juegos, {eval_time:.0f}s) ---")
        print(f"    Nuevo:  win_rate={100*wr_new_pol:.1f}%  avg_score={avg_new_pol:.1f}")
        print(f"    Mejor:  win_rate={100*wr_best_pol:.1f}%  avg_score={avg_best_pol:.1f}")

        # ------------------------------------------------
        # 4. CHAMPION UPDATE
        # ------------------------------------------------
        # Comparar por score promedio (mas granular que win rate para scores bajos)
        if avg_new_pol > avg_best_pol * cfg.accept_threshold / 0.5:
            print(f"  >>> NUEVO CHAMPION aceptado! <<<")
            best_net.load_state_dict(net.state_dict())
            best_win_rate = max(best_win_rate, wr_new_pol)
        else:
            print(f"  Mejor modelo retenido.")
            net.load_state_dict(best_net.state_dict())

        # ------------------------------------------------
        # 5. RESUMEN ITERACION
        # ------------------------------------------------
        iter_time = time.time() - iter_start
        total_elapsed = time.time() - training_start
        iters_done = iteration - start_iter + 1
        iters_left = cfg.max_iterations - iteration - 1
        eta_sec = (iter_time * iters_left) if iters_left > 0 else 0
        eta_min = eta_sec / 60

        print(f"  --- Resumen iter {iteration+1}/{cfg.max_iterations} ---")
        print(f"    Tiempos: self-play={sp_time:.0f}s  train={train_time:.0f}s  eval={eval_time:.0f}s  total={iter_time:.0f}s")
        print(f"    Acumulado: {total_elapsed/60:.1f} min | Restantes: {iters_left} iters | ETA ~{eta_min:.0f} min")
        print(f"    Buffer: {len(buffer)} posiciones | Mejor win_rate (policy): {best_win_rate:.3f}")
        if (iteration + 1) % cfg.checkpoint_interval == 0:
            save_checkpoint(net, best_net, optimizer, iteration + 1, best_win_rate, cfg, buffer)
            print(f"    Checkpoint guardado (iter {iteration+1})")

    print(f"\n{'='*60}")
    print(f"  ENTRENAMIENTO COMPLETADO")
    print(f"  Mejor win rate (policy): {best_win_rate:.3f}")
    print(f"{'='*60}")

    return best_net

print("Loop de entrenamiento definido. Ejecutar la siguiente celda para entrenar.")

In [ ]:
# ============================================================
# Cell X: Benchmark rapido de throughput (opcional)
# ============================================================

def benchmark_train_epoch(cfg, n_batches=80):
    """
    Benchmark sintetico del throughput de entrenamiento.
    Mide samples/s usando train_epoch sobre un buffer artificial.
    """
    net_bench = AlphaSnakeNet(4, cfg.net_blocks, cfg.net_channels, cfg.board_size).to(device)
    optimizer_bench = torch.optim.Adam(net_bench.parameters(), lr=cfg.lr, weight_decay=cfg.weight_decay)
    scaler_bench = torch.amp.GradScaler(device="cuda", enabled=bool(getattr(cfg, "use_amp", True) and device.type == "cuda"))

    # Llenar buffer con datos sinteticos para medir solo pipeline de train.
    rb = ReplayBuffer(max(cfg.batch_size * (n_batches + 2), cfg.batch_size * 16))
    synth_states = np.random.randn(cfg.batch_size, 4, cfg.board_size, cfg.board_size).astype(np.float32)
    synth_policies = np.full((cfg.batch_size, 4), 0.25, dtype=np.float32)
    synth_values = np.random.uniform(-1, 1, size=(cfg.batch_size,)).astype(np.float32)

    block = list(zip(synth_states, synth_policies, synth_values))
    for _ in range(n_batches + 4):
        rb.add(block)

    cfg_bench = dataclasses.replace(cfg)
    cfg_bench.batch_size = cfg.batch_size

    if device.type == "cuda":
        torch.cuda.synchronize()
    t0 = time.time()

    for _ in range(n_batches):
        train_epoch(net_bench, optimizer_bench, rb, cfg_bench, device, scaler=scaler_bench)

    if device.type == "cuda":
        torch.cuda.synchronize()
    dt = time.time() - t0

    total_samples = n_batches * max(1, len(rb) // cfg_bench.batch_size) * cfg_bench.batch_size
    samples_per_sec = total_samples / max(dt, 1e-6)
    print(f"[Benchmark train] {samples_per_sec:.1f} samples/s | tiempo={dt:.2f}s | batch={cfg.batch_size} | AMP={getattr(cfg, 'use_amp', True)}")


# Uso sugerido (opcional):
# benchmark_train_epoch(config, n_batches=40)


In [ ]:
# ============================================================
# Cell 10: EJECUTAR ENTRENAMIENTO
# ============================================================
# Esta celda lanza el entrenamiento completo.
# Se puede interrumpir y re-ejecutar: los checkpoints se guardan en save_dir (Drive/workspace).

best_model = train_alphasnake(config)

## Evaluacion Final y Exportacion ONNX

Evaluar el mejor modelo con MCTS completo y exportar a ONNX para usar en el navegador.

In [ ]:
# ============================================================
# Cell 11: Evaluacion Final con MCTS
# ============================================================
# Cargar el mejor modelo (en caso de re-ejecucion)
best_model_path = os.path.join(config.save_dir, 'best_model.pt')
if os.path.exists(best_model_path):
    best_model = AlphaSnakeNet(4, config.net_blocks, config.net_channels, config.board_size).to(device)
    best_model.load_state_dict(torch.load(best_model_path, map_location=device))
    print("Mejor modelo cargado desde save_dir.")

# Evaluacion completa con MCTS
print("\nEvaluacion final con MCTS (esto puede tardar)...")
eval_cfg = Config(
    board_size=config.board_size,
    max_steps=config.max_steps,
    num_simulations=config.num_simulations,
    c_puct=config.c_puct,
    dir_alpha=config.dir_alpha,
    dir_eps=0.0,  # Sin noise en evaluacion
    food_samples=config.food_samples,
)

wr, avg_score, scores = evaluate_mcts(best_model, eval_cfg, device, n_games=100)
print(f"\nResultados finales ({100} juegos con MCTS):")
print(f"  Win rate: {wr:.3f} ({int(wr*100)}/100)")
print(f"  Avg score: {avg_score:.1f} / {config.board_size**2 - 3}")
print(f"  Min score: {min(scores)}, Max score: {max(scores)}")
print(f"  Median score: {np.median(scores):.0f}")

In [ ]:
# ============================================================
# Cell 12: Exportar a ONNX
# ============================================================
import onnx

best_model.eval()
best_model.cpu()

# Dummy input: (batch=1, channels=4, h=10, w=10)
dummy_input = torch.randn(1, 4, config.board_size, config.board_size)

# Path: un solo archivo en save_dir (Colab, Vast.ai y local)
onnx_path = os.path.join(config.save_dir, 'alphasnake.onnx')

# Exportar
torch.onnx.export(
    best_model,
    dummy_input,
    onnx_path,
    export_params=True,
    opset_version=13,
    do_constant_folding=True,
    input_names=['state'],
    output_names=['policy', 'value'],
    dynamic_axes={
        'state': {0: 'batch_size'},
        'policy': {0: 'batch_size'},
        'value': {0: 'batch_size'},
    }
)

# Verificar
model_onnx = onnx.load(onnx_path)
onnx.checker.check_model(model_onnx)
print(f"Modelo ONNX exportado y verificado: {onnx_path}")

# Verificar que las salidas coincidan
import onnxruntime as ort

ort_session = ort.InferenceSession(onnx_path)
test_input = np.random.randn(1, 4, 10, 10).astype(np.float32)

# PyTorch
best_model.cpu()
with torch.no_grad():
    pt_policy, pt_value = best_model(torch.from_numpy(test_input))
pt_policy = pt_policy.numpy()
pt_value = pt_value.numpy()

# ONNX
ort_inputs = {'state': test_input}
ort_policy, ort_value = ort_session.run(None, ort_inputs)

print(f"\nVerificacion PyTorch vs ONNX:")
print(f"  Policy diff max: {np.abs(pt_policy - ort_policy).max():.8f}")
print(f"  Value diff max:  {np.abs(pt_value - ort_value).max():.8f}")

# Tamanio
size_mb = os.path.getsize(onnx_path) / (1024 * 1024)
print(f"Tamanio del modelo: {size_mb:.2f} MB")

In [ ]:
# ============================================================
# Cell 13: Ubicación del modelo ONNX
# ============================================================
# El modelo está en config.save_dir. En Colab descomenta: from google.colab import files; files.download(onnx_path)
# En Vast.ai: descarga por Jupyter (clic derecho en alphasnake.onnx) o por scp.
print(f"Modelo ONNX guardado en: {onnx_path}")
print("Descarga iniciada. Coloca el archivo alphasnake.onnx en games/snake/ai/")